## Navigators
Navigators provide the ability to drill into the result set providing a summary of the distribution.  They are particularly useful when you are interested in gathering the domain of values for a particular Property.  Navigators can be used against a specific View, used in conjunction with either a query, a filter or both.  Navigators can be simple or very powerful, but provide a very useful way to capture results in logical buckets.  

In [1]:
import refinitiv.data as rd
from refinitiv.data.content import search
import pandas as pd
import datetime as dt
import dateutil.relativedelta

# Default session - desktop
rd.open_session()

<refinitiv.data.session.Definition object at 0x1eafe7f12b0 {name='default'}>

In [2]:
pd.set_option('display.max_colwidth', 140)
rd.__version__

'1.0.0b9'

#### Example - Property values
The most basic usage of navigators is to determine the available values within a given Property.  For example, when searching for assets, we may want to narrow down the specific country an asset is traded in.  We can determine the available countries to work with.

In [3]:
# The following will list the countries assets are listed.  We can use this output to filter based on a specific country.
response = search.Definition(
    top = 0,
    navigators = "RCSCountryLeaf(buckets:20)"   # Show the top 20
).get_data()
response.data.raw["Navigators"]["RCSCountryLeaf"]["Buckets"]

[{'Label': 'United States', 'Count': 13029615},
 {'Label': 'Eurobond', 'Count': 4290309},
 {'Label': 'China (Mainland)', 'Count': 1297730},
 {'Label': 'Colombia', 'Count': 1085611},
 {'Label': 'South Korea', 'Count': 571896},
 {'Label': 'Germany', 'Count': 486688},
 {'Label': 'Canada', 'Count': 415803},
 {'Label': 'France', 'Count': 252540},
 {'Label': 'India', 'Count': 196980},
 {'Label': 'Japan', 'Count': 164874},
 {'Label': 'Italy', 'Count': 134969},
 {'Label': 'Chile', 'Count': 132477},
 {'Label': 'Australia', 'Count': 106668},
 {'Label': 'Brazil', 'Count': 68895},
 {'Label': 'Switzerland', 'Count': 67077},
 {'Label': 'Spain', 'Count': 62895},
 {'Label': 'Austria', 'Count': 58537},
 {'Label': 'Norway', 'Count': 57748},
 {'Label': 'Russia', 'Count': 56828},
 {'Label': 'United Kingdom', 'Count': 55878}]

#### Example - Vessels
If I want to perform a basic search for tankers and capture the Hull Types, I could start with the following basic query:

In [4]:
response=search.Definition(
    view = search.SearchViews.VESSEL_PHYSICAL_ASSETS,
    query = "tanker",
    select = "DocumentTitle, HullType",
    top = 10000
).get_data()
response.data.df

,DocumentTitle,HullType
0,"MOZAH, LNG Tanker, NAKILAT SHIPPING QATAR LTD, Red Sea|Aden Tanker Zone",Single Hull
1,"UMM SLAL, Liquefied Natural Gas; LNG Tanker, NAKILAT SHIPPING QATAR LTD, Gulf of Aden|Aden Tanker Zone",Single Hull
2,"BU SAMRA, Liquefied Natural Gas; LNG Tanker, NAKILAT SHIPPING QATAR LTD, East China Sea|Shanghai Tanker Zone",Single Hull
3,"AL MAYEDA, LNG Tanker, STASCO SHIP MANAGEMENT, Borneo Sea|Singapore Tanker Zone",Single Hull
4,"MEKAINES, LNG Tanker, NAKILAT SHIPPING QATAR LTD, Gulf of Aden|Aden Tanker Zone",Single Hull
...,...,...
9995,"YU HU 3502, Other Tanker, Shanghai Tanker Zone|Yangtze River",<NA>
9996,"PILATUS, Other Tanker, UK Continent Tanker Zone|River Rhine",<NA>
9997,"FENG LONG YOU 99, Other Tanker, Jiangsu|Shanghai Tanker Zone|Yangtze River|China (Mainland)",<NA>
9998,"WAN TONG DA 55, Other Tanker, Shanghai Tanker Zone|Yangtze River",<NA>


Here, we can see a list of the different tankers and their Hull Types. Within this collection, while I can clearly see there are different Hull Types list, i.e. "Single", "Double", "NA", etc - I'm also seeing duplicates.  Instead of going through the effort to process the above result to pull out the unique list of Hull Types, I will instead use a Navigator - which was designed to do this work for me.

Using the above query, I will apply a Navigator by selecting the 'HullType' Property.  What this does is instruct search to bucket all unique Hull Types and summarize the total found for each. To capture the results, I will need to utilize the Content Layer API call.

In [5]:
response=search.Definition(
    view = search.SearchViews.VESSEL_PHYSICAL_ASSETS,
    query = "tanker",
    top = 0,       # I'm not interested in the hits, only details within the navigator response section
    navigators = "HullType"
).get_data()
response.data.raw["Navigators"]["HullType"]["Buckets"]

[{'Label': 'Double Hull', 'Count': 16700},
 {'Label': 'Single Hull', 'Count': 5429}]

#### Example - Listing of exchanges per asset category
In the following example, I'm going to list the top 3 asset types in Canada and for each, display the top 2 exchanges trading on those assets.

In [6]:
response=search.Definition(
    view = search.SearchViews.EQUITY_QUOTES,
    filter = "AssetState eq 'AC' and RCSExchangeCountryLeaf eq 'Canada'",    # Filter on all active (AC) assets within Canada
    top = 0,
    navigators = "RCSAssetCategoryLeaf(buckets:3, sub:ExchangeName(buckets:2))"  #  For the top 3 categories, show the top 2 exchanges
).get_data()

In [7]:
# Pretty display of the listing
rank = 1
for asset in response.data.raw["Navigators"]["RCSAssetCategoryLeaf"]["Buckets"]:
    print(f'{rank}. {asset["Label"]}')
    exchangeRank = 1
    for exchange in asset["ExchangeName"]["Buckets"]:
        print(f'\t{exchangeRank}. {exchange["Label"]} ({exchange["Count"]})')
        exchangeRank += 1
    rank += 1

1. Equity Cash Option
	1. Montreal Options Exchange (104749)
	2. Refinitiv (46984)
2. Ordinary Share
	1. Aequitas NEO-L (3522)
	2. Aequitas NEO-N (3521)
3. Equity Future Spread
	1. Montreal Exchange (5629)
	2. Montreal Options Exchange (17)


#### Example - Outstanding Government debt in the last 3 months.
In this example, I will be using some nested expressions to summarize categories within the buckets I collect.  Specifically, I'm looking for government debt ('govt') issued any time after 3-months ago using the following filter:
- "DBType eq 'govt' and IssueDate gt today-3months" (DBType identifies government debt)
    
Based on the result set, I want categorize or create buckets (top 5) based on the domicile of the issuer.  For each domicile, sum the outstanding debt and also include the maximum coupon rate for each one.
- RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)

In [8]:
# Get date N months ago
months = 3
date = dt.datetime.today() - dateutil.relativedelta.relativedelta(months=months)

In [9]:
# Retrieve the top 5 outstanding government debt in the last 3 months
response = search.Definition(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    filter = "DbType eq 'govt' and IssueDate gt " + date.strftime("%Y-%m-%d"),
    top = 0,
    navigators = "RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)"
).get_data()
response.data.raw["Navigators"]["RCSDomicileLeaf"]["Buckets"]

[{'Label': 'United States',
  'Count': 75,
  'max_CouponRate': 3.0,
  'sum_FaceOutstandingUSD': 2262671339900.0},
 {'Label': 'Japan',
  'Count': 56,
  'max_CouponRate': 1.0,
  'sum_FaceOutstandingUSD': 907811384735.0},
 {'Label': 'China (Mainland)',
  'Count': 33,
  'max_CouponRate': 3.52,
  'sum_FaceOutstandingUSD': 249204462656.0},
 {'Label': 'Singapore',
  'Count': 45,
  'max_CouponRate': 1.43,
  'sum_FaceOutstandingUSD': 172137115955.0},
 {'Label': 'Hong Kong',
  'Count': 40,
  'max_CouponRate': 1.79,
  'sum_FaceOutstandingUSD': 122408149130.0}]

In [10]:
rd.close_session()